# Neuarl Network Regressor in TensorFlow

In [3]:
import os
os.chdir("D:/ML_Projects/MercedesBenz-Kaggle/")

import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
np.random.seed(12548)

def read_data():
	train_data = pd.read_csv("./data/train.csv")
	test_data = pd.read_csv("./data/test.csv")

	return train_data, test_data


def process_data(train_data, test_data):
	binary_cols, all_zero_cols, all_one_cols = [],[],[]
	for col in train_data.iloc[:,10:]:
		unique_vals = train_data[col].unique()
		if np.array_equal(unique_vals, [1,0]) or np.array_equal(unique_vals, [0,1]):
			binary_cols.append(col)
		elif np.array_equal(unique_vals, [0]):
			all_zero_cols.append(col)
		elif np.array_equal(unique_vals, [1]):
			all_one_cols.append(col)
		else:
			print(unique_vals)

	# Drop columns with only zeros
	train_data = train_data.drop(all_zero_cols, axis=1)
	test_data = test_data.drop(all_zero_cols, axis=1)

	train_cat_cols = train_data.iloc[:,2:10]
	test_cat_cols = test_data.iloc[:,1:9]
	freq=[]
	col_names = []
	cat_mismatch = []

	for train_col, test_col in zip(train_cat_cols, test_cat_cols):
		col_names.append(train_col)
		train_freq = len(train_cat_cols[train_col].unique())
		test_freq = len(test_cat_cols[test_col].unique())

		if train_freq!=test_freq:
			cat_mismatch.append(train_col)

		freq.append([train_freq, test_freq])
	freq = pd.DataFrame(freq, columns=['Train_Freq', 'Test_Freq'], index=col_names)

	train_data = train_data.drop(cat_mismatch, axis=1)
	test_data = test_data.drop(cat_mismatch, axis=1)

	return train_data, test_data


def prepare_data_ml(train_data, test_data):
	X_train = pd.get_dummies(train_data)
	X_train = X_train.drop(['ID','y'], axis=1).values
	y_train = train_data.y.values

	X_test = pd.get_dummies(test_data)
	y_test_id = test_data.ID.values
	X_test = X_test.drop(['ID'], axis=1).values

	return X_train, y_train, X_test, y_test_id

In [7]:
def input_fn(data):
    feature_cols = [tf.contrib.layers.real_valued_column_valued_column(k) for k in data[0].columns]
    labels = data[1]
    return feature_cols, labels

train_data, test_data = read_data()
train_data, test_data = process_data(train_data, test_data)
X_train, y_train, X_test, y_test_id = prepare_data_ml(train_data, test_data)

In [8]:
feature_cols, labels = input_fn([X_train, y_train])

AttributeError: 'numpy.ndarray' object has no attribute 'columns'